In [8]:
!pip install pyspark

In [21]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder.appName("Lab12").getOrCreate()

In [12]:
from pyspark.ml.linalg import Vectors as vectors
from pyspark.ml.stat import Correlation as correlation

data= [(vectors.sparse(4, [(0, 1.0), (3, -2.0)]),), (vectors.dense ([4.0, 5.0, 0.0, 3.0]),), (vectors.dense ([6.0, 7.0, 0.0, 8.0]),), (vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)]

df =spark.createDataFrame(data,["features"])

r1 = correlation.corr (df, "features").head()

print("Pearson correlation matrix:\n" + str(r1[0]))

r2= correlation.corr(df, "features", "spearman").head()

print("Spearman correlation matrix:\n"+ str(r2[0]))

Pearson correlation matrix:
DenseMatrix([[1.        , 0.05564149,        nan, 0.40047142],
             [0.05564149, 1.        ,        nan, 0.91359586],
             [       nan,        nan, 1.        ,        nan],
             [0.40047142, 0.91359586,        nan, 1.        ]])
Spearman correlation matrix:
DenseMatrix([[1.        , 0.10540926,        nan, 0.4       ],
             [0.10540926, 1.        ,        nan, 0.9486833 ],
             [       nan,        nan, 1.        ,        nan],
             [0.4       , 0.9486833 ,        nan, 1.        ]])


In [15]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest

data = [(0.0, Vectors.dense(0.5, 10.0)),
        (0.0, Vectors.dense(1.5, 20.0)),
        (1.0, Vectors.dense(1.5, 30.0)),
        (0.0, Vectors.dense(3.5, 30.0)),
        (0.0, Vectors.dense(3.5, 40.0)),
        (1.0, Vectors.dense(3.5, 40.0))]

df = spark.createDataFrame(data, ["label", "features"])

r = ChiSquareTest.test(df, "features", "label").head()

print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))


pValues: [0.6872892787909721,0.6822703303362126]
degreesOfFreedom: [2, 3]
statistics: [0.75,1.5]


In [18]:
spark.stop()
from pyspark import SparkContext

# Create a SparkContext object
sc = SparkContext("local", "example")


In [24]:
from pyspark.sql import SparkSession
from pyspark.ml.stat import Summarizer
from pyspark.ml.linalg import Vectors

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Example") \
    .getOrCreate()

# Create an RDD
rdd = sc.parallelize([
    (1.0, Vectors.dense(1.0, 1.0, 1.0)),
    (0.0, Vectors.dense(1.0, 2.0, 3.0))
])

# Convert RDD to DataFrame
df = spark.createDataFrame(rdd, ["weight", "features"])

# Create summarizer for multiple metrics "mean" and "count"
summarizer = Summarizer.metrics("mean", "count")

# Compute statistics for multiple metrics with weight
df.select(summarizer.summary(df.features, df.weight)).show(truncate=False)

# Compute statistics for multiple metrics without weight
df.select(summarizer.summary(df.features)).show(truncate=False)

# Compute statistics for single metric "mean" with weight
df.select(Summarizer.mean(df.features, df.weight)).show(truncate=False)

# Compute statistics for single metric "mean" without weight
df.select(Summarizer.mean(df.features)).show(truncate=False)

# Stop the SparkSession
spark.stop()


+-----------------------------------+
|aggregate_metrics(features, weight)|
+-----------------------------------+
|{[1.0,1.0,1.0], 1}                 |
+-----------------------------------+

+--------------------------------+
|aggregate_metrics(features, 1.0)|
+--------------------------------+
|{[1.0,1.5,2.0], 2}              |
+--------------------------------+

+--------------+
|mean(features)|
+--------------+
|[1.0,1.0,1.0] |
+--------------+

+--------------+
|mean(features)|
+--------------+
|[1.0,1.5,2.0] |
+--------------+

